<a href="https://colab.research.google.com/github/hwangdonggeun/project/blob/main/date.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import requests
import json
from datetime import datetime
import torch
import torch.nn as nn
import torch.optim as optim

API_KEY = "b528b386b4962e0acde62d7931d3e58e"
RECORD_FILE = "date_course_records.json"

KOREAN_CITIES = [
    "Seoul", "Busan", "Incheon", "Daegu", "Daejeon", "Gwangju", "Suwon", "Ulsan", "Jeonju", "Cheongju",
    "Changwon", "Gyeongju", "Jeju", "Pohang", "Gimhae", "Hwaseong", "Anyang", "Seongnam", "Goyang", "Yongin"
]

activity_sub = {
    "관람": ["영화 보기", "전시회 관람", "뮤지컬 관람", "공연 관람", "미술관 방문"],
    "액티비티": ["등산하기", "실내암벽등반", "카트레이싱", "자전거 타기", "서핑"],
    "실내": ["방탈출", "보드게임카페", "노래방", "만화카페", "VR체험"]
}

cuisine_sub = {
    "한식": ["순대국밥", "불고기", "제육볶음", "된장찌개", "칼국수"],
    "중식": ["짜장면", "짬뽕", "마파두부", "마라탕", "딤섬"],
    "양식": ["파스타", "피자", "스테이크", "리조또", "햄버거"],
    "일식": ["초밥", "라멘", "돈카츠", "우동", "규동"]
}

def map_weather_to_id(weather_str):
    weather_map = {
        "Clear": 0,
        "Clouds": 1,
        "Rain": 2,
        "Snow": 3,
        "Drizzle": 4,
        "Thunderstorm": 5,
        "Mist": 6,
        "Fog": 7,
        "Haze": 8,
        "Smoke": 9,
    }
    return weather_map.get(weather_str, 1)

class RecommendationModel(nn.Module):
    def __init__(self, activity_size, cuisine_size, weather_size, hidden_size=32):
        super(RecommendationModel, self).__init__()
        self.activity_embed = nn.Embedding(activity_size, hidden_size)
        self.cuisine_embed = nn.Embedding(cuisine_size, hidden_size)
        self.weather_embed = nn.Embedding(weather_size, hidden_size)

        self.fc1 = nn.Linear(hidden_size * 3, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.output = nn.Linear(hidden_size, activity_size)

    def forward(self, activity, cuisine, weather):
        activity_emb = self.activity_embed(activity)
        cuisine_emb = self.cuisine_embed(cuisine)
        weather_emb = self.weather_embed(weather)

        x = torch.cat([activity_emb, cuisine_emb, weather_emb], dim=1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        output = self.output(x)
        return output

def get_user_preferences():
    print("오늘 데이트는 어떻게 할까?")
    print("날씨를 확인할 도시 이름을 영어로 입력해줘.")
    print("한국 도시 예시: Seoul, Busan, Incheon, Daegu, Daejeon 등")
    city = input("어느 도시에 갈거야?: ")
    retry_count = 0
    while city not in KOREAN_CITIES:
        retry_count += 1
        if retry_count > 3:
            print("도시 입력에 여러 번 실패했어. 기본값(Seoul)으로 할게.")
            city = "Seoul"
            break
        print("입력한 도시가 목록에 없어. 다시 입력해줘.")
        city = input("어느 도시에 갈거야?: ")

    activity = input("오늘 뭐할래? (예: 관람, 액티비티, 실내): ").strip()
    if activity not in ["관람", "액티비티", "실내"]:
        print("유효하지 않은 활동이므로 관람을 추천할게..")
        activity = "관람"

    cuisine = input("뭐 먹을래? (예: 한식, 중식, 양식, 일식): ").strip()
    if cuisine not in ["한식", "중식", "양식", "일식"]:
        print("유효하지 않은 음식이므로 한식을 추천할게.")
        cuisine = "한식"

    return city, activity, cuisine

def get_weather(city):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    try:
        response = requests.get(url)
        response.raise_for_status()
        weather_data = response.json()
        main_weather = weather_data['weather'][0]['main']
        temperature = weather_data['main']['temp']
        print(f"오늘의 날씨!: {main_weather}, 온도: {temperature}°C")
        return main_weather, temperature
    except requests.RequestException as e:
        print("날씨 정보를 가져오는 중 오류가 발생했어 ㅠㅠ:", e)
        return None, None

def train_model(model, data, epochs=10, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        total_loss = 0
        for activity, cuisine, weather, target in data:
            activity = torch.tensor([activity], dtype=torch.long)
            cuisine = torch.tensor([cuisine], dtype=torch.long)
            weather = torch.tensor([weather], dtype=torch.long)
            target = torch.tensor([target], dtype=torch.long)

            optimizer.zero_grad()
            output = model(activity, cuisine, weather)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss:.4f}")

def predict(model, activity, cuisine, weather):
    model.eval()
    activity = torch.tensor([activity], dtype=torch.long)
    cuisine = torch.tensor([cuisine], dtype=torch.long)
    weather = torch.tensor([weather], dtype=torch.long)

    with torch.no_grad():
        output = model(activity, cuisine, weather)
        predicted_class = output.argmax(dim=1).item()
    return predicted_class

def load_records():
    try:
        with open(RECORD_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    except FileNotFoundError:
        return []
    except json.JSONDecodeError:
        print("기록 파일 포맷 에러. 빈 리스트 반환")
        return []

def save_records(records):
    with open(RECORD_FILE, "w", encoding="utf-8") as f:
        json.dump(records, f, ensure_ascii=False, indent=4)

def save_recommendation(record):
    records = load_records()
    records.append(record)
    save_records(records)

def prepare_training_data():
    records = load_records()
    if not records:
        print("추천 기록이 없습니다.")
        return []

    activity_map = {"관람": 0, "액티비티": 1, "실내": 2}
    cuisine_map = {"한식": 0, "중식": 1, "양식": 2, "일식": 3}

    data = []
    for record in records:
        activity = activity_map.get(record.get("activity", "관람"), 0)
        cuisine = cuisine_map.get(record.get("cuisine", "한식"), 0)
        weather_str = record.get("weather", "Clouds")
        weather_id = map_weather_to_id(weather_str)
        target = activity
        data.append((activity, cuisine, weather_id, target))
    return data

def recommend_course_with_model(model, activity, cuisine, weather_str):
    activity_map = {"관람": 0, "액티비티": 1, "실내": 2}
    cuisine_map = {"한식": 0, "중식": 1, "양식": 2, "일식": 3}
    reverse_activity_map = {v: k for k, v in activity_map.items()}
    reverse_cuisine_map = {v: k for k, v in cuisine_map.items()}

    activity_id = activity_map.get(activity, 0)
    cuisine_id = cuisine_map.get(cuisine, 0)
    weather_id = map_weather_to_id(weather_str if weather_str else "Clouds")

    predicted = predict(model, activity_id, cuisine_id, weather_id)
    recommended_activity = reverse_activity_map.get(predicted, "관람")
    recommended_cuisine = cuisine
    chosen_activity_item = random.choice(activity_sub[recommended_activity])
    chosen_cuisine_item = random.choice(cuisine_sub[recommended_cuisine])

    print(f"추천 코스: {chosen_activity_item} + {chosen_cuisine_item}")
    return recommended_activity, recommended_cuisine, chosen_activity_item, chosen_cuisine_item

def recommend_course_dl_only(model):
    activity_list = ["관람", "액티비티", "실내"]
    cuisine_list = ["한식", "중식", "양식", "일식"]
    rand_activity = random.choice(activity_list)
    rand_cuisine = random.choice(cuisine_list)
    weather_str = "Clouds"
    activity_map = {"관람": 0, "액티비티": 1, "실내": 2}
    cuisine_map = {"한식": 0, "중식": 1, "양식": 2, "일식": 3}
    reverse_activity_map = {v: k for k, v in activity_map.items()}

    activity_id = activity_map.get(rand_activity, 0)
    cuisine_id = cuisine_map.get(rand_cuisine, 0)
    weather_id = map_weather_to_id(weather_str)

    predicted = predict(model, activity_id, cuisine_id, weather_id)
    recommended_activity = reverse_activity_map.get(predicted, "관람")
    recommended_cuisine = rand_cuisine

    chosen_activity_item = random.choice(activity_sub[recommended_activity])
    chosen_cuisine_item = random.choice(cuisine_sub[recommended_cuisine])

    print("\n[딥러닝 추천 결과]")
    print(f"추천된 활동 대분류: {recommended_activity}")
    print(f"추천된 음식 대분류: {recommended_cuisine}")
    print(f"추천 코스: {chosen_activity_item} + {chosen_cuisine_item}\n")

if __name__ == "__main__":
    activity_size = 3
    cuisine_size = 4
    weather_size = 10

    model = RecommendationModel(activity_size, cuisine_size, weather_size)

    training_data = prepare_training_data()
    if training_data:
        print("모델 학습 중...")
        train_model(model, training_data)

    while True:
        print("1. 추천받기 (사용자 입력 기반)")
        print("2. 딥러닝 추천 결과 보기 (랜덤)")
        print("3. 종료")
        choice = input("원하는 작업을 선택하세요: ").strip()

        if choice == "1":
            city, activity, cuisine = get_user_preferences()
            weather_str, temperature = get_weather(city)

            if weather_str and temperature is not None:
                print(f"현재 {city}의 날씨는 {weather_str}, 온도는 {temperature}°C입니다.")
            else:
                print("날씨 정보를 가져올 수 없어 기본 추천을 가져올게 ㅠㅠ")
                weather_str = "Clouds"
            recommended_activity, recommended_cuisine, chosen_activity_item, chosen_cuisine_item = \
                recommend_course_with_model(model, activity, cuisine, weather_str)

            while True:
                like = input("마음에 드나요? (예/아니요): ").strip()
                if like == "예":
                    print("좋아해줘서 고마워!")
                    break
                elif like == "아니요":
                    activities = list(activity_sub.keys())
                    cuisines = list(cuisine_sub.keys())
                    rand_activity = random.choice(activities)
                    rand_cuisine = random.choice(cuisines)
                    recommended_activity, recommended_cuisine, chosen_activity_item, chosen_cuisine_item = \
                        recommend_course_with_model(model, rand_activity, rand_cuisine, weather_str)
                else:
                    print("예 또는 아니요로 답변해주세요.")

        elif choice == "2":
            recommend_course_dl_only(model)

        elif choice == "3":
            print("프로그램을 종료합니다.")
            break

        else:
            print("잘못된 입력입니다. 다시 선택해주세요.")
